In [13]:
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, RobertaModel, get_linear_schedule_with_warmup
from utils import *
from datasets import load_dataset
import traceback
import asyncio
import os
import warnings
import re


os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

"""GLOBAL PARAMS"""
MAX_LEN = 180
TEST_SIZE = 0.2
RANDOM_STATE = 42
EPOCHS_KFOLD_FROZEN = 100
LEARNING_RATE_KFOLD_FROZEN = 3e-4
K_FOLDS_FROZEN = 2
BATCH_SIZE = 16
LOSS_FUNCTION = nn.CrossEntropyLoss()

"DIRECTORIES"
SAVE_DIRECTORY_FROZEN = "./GLUE/RoBERTa_entrenado_kfold"
SAVE_DIRECTORY_UNFROZEN = "./GLUE/RoBERTa_entrenado_kfold_unfrozen_last_layer"

TYPED_STORAGE_WARNING = re.compile(".TypedStorage is deprecated.")
FALLBACK_KERNEL_WARNING = re.compile(".Using FallbackKernel: aten.cumsum.")
TRITON_RANDOM_WARNING = re.compile(".using triton random, expect difference from eager.")

def warning_filter(message, category, filename, lineno, file=None, line=None):
    if category == UserWarning and (TYPED_STORAGE_WARNING.match(str(message)) or FALLBACK_KERNEL_WARNING.match(str(message)) or TRITON_RANDOM_WARNING.match(str(message))):
        return False
    return True


warnings.showwarning = warning_filter
warnings.filterwarnings("ignore", category=UserWarning, module="torch._inductor.ir")



# Load the GLUE dataset
def load_glue_dataset(task_name):
    dataset = load_dataset('glue', task_name)
    return dataset

# Preprocess the GLUE dataset
def preprocess_glue_data(dataset, tokenizer, max_len):
    def encode(example):
        return tokenizer(example['sentence'], truncation=True, padding='max_length', max_length=max_len)

    encoded_dataset = dataset.map(encode, batched=True)
    return encoded_dataset




In [14]:
def load_glue_dataset(task_name):
    dataset = load_dataset('glue', task_name)
    return dataset

# Preprocess the GLUE dataset
def preprocess_glue_data(dataset, tokenizer, max_len):
    def encode(example):
        return tokenizer(example['sentence'], truncation=True, padding='max_length', max_length=max_len)

    encoded_dataset = dataset.map(encode, batched=True)
    return encoded_dataset

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

task_name = 'sst2'  # Choose the task name from the GLUE dataset
glue_dataset = load_glue_dataset(task_name)
train_dataset = glue_dataset['train']

# Set max sequence length and load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Tokenize and encode the input data
encoded_train_dataset = preprocess_glue_data(train_dataset, tokenizer, MAX_LEN)

# Encode the emotion labels using LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(encoded_train_dataset['label'])


cuda


HfHubHTTPError: 504 Server Error: Gateway Time-out for url: https://huggingface.co/api/datasets/glue